In [1]:
import hail as hl
hl.init(min_block_size=256)
import os
import pandas as pd
import subprocess
os.environ['PYSPARK_SUBMIT_ARGS'] = "--driver-memory 700G pyspark-shell"
# !module load java/java-1.8
gs = "/gstock/biolo_datasets/variation/gnomAD/r2.1.1/vcf/exomes/gnomad.exomes.r2.1.1.sites.ht/"
# gs = "/data/scratch/gnomAD/v3/gnomad.genomes.v3.1.sites.ht/"
data = hl.read_table(gs)
data

Running on Apache Spark version 2.4.1
SparkUI available at http://vega.local:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.64-1ef70187dc78
LOGGING: writing to /enadisk/maison/sage2016/weber/PycharmProjects/EXOTIC/notebooks/Hail/hail-20210422-1840-0.2.64-1ef70187dc78.log


In [2]:
exotic = pd.read_parquet("/gstock/EXOTIC/data/EXOTIC/EXOTIC_modified_zscore_mean_prop_corrected_with_variable.parquet")
biomart = pd.read_csv('/gstock/EXOTIC/data/OTHERS/biomart_refseq_ensembl_hgnc.txt.gz', compression='gzip', sep='\t')
exotic = pd.merge(biomart[['Gene name', 'Chromosome/scaffold name']].drop_duplicates().rename({'Gene name' : 'Gene', 'Chromosome/scaffold name' : 'CHROM'}, axis=1), exotic, on=['Gene'])
exotic['CHROM'] = exotic['CHROM'].astype(str)

exotic = exotic.loc[~exotic['CHROM'].str.contains('HSCHR|HG')]
exotic = exotic[['CHROM', 'CDS_start', 'CDS_stop', 'MAP']].sort_values(by=['CHROM', 'CDS_start'])
exotic.to_csv('/biolo/ngs/remap/EXOTIC.bed', header=False, sep='\t', index=False)

/home/weber/.conda/envs/hail/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
bed_file = hl.import_bed('/biolo/ngs/remap/EXOTIC.bed', reference_genome='GRCh37')
bed_file = bed_file.head(100)
# bed_file.show()

2021-04-22 18:48:44 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
  Loading field 'f3' as type str (user-supplied)


In [55]:
data_lite = data.filter(hl.is_defined(bed_file[data.locus]))
data_lite = data_lite.filter(data_lite.variant_type == 'snv')
data_lite = data_lite.filter(data_lite.freq[0]['AC'] > 0)
# data_lite['AF'] = data_lite.freq[0]['AC']
data_lite = data_lite.annotate(
    MAP = bed_file[data_lite.locus].target,
    AF = data_lite.freq[0]['AF'],
    id = hl.str(data_lite.vep.id).replace('/', '_')
)
data_lite = data_lite.select(data_lite.AF, data_lite.rsid, data_lite.variant_type, data_lite.vep.end, data_lite.id, data_lite.vep.most_severe_consequence, data_lite.MAP)
data_lite = data_lite.key_by('id')

In [49]:
data_lite.head(10).show()

2021-04-22 19:09:51 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-04-22 19:09:52 Hail: INFO: Ordering unsorted dataset with network shuffle


+---------------+------------+
| locus         | alleles    |
+---------------+------------+
| locus<GRCh37> | array<str> |
+---------------+------------+
| 1:906494      | ["G","A"]  |
| 1:906496      | ["C","T"]  |
| 1:906497      | ["G","A"]  |
| 1:906503      | ["C","A"]  |
| 1:906505      | ["C","T"]  |
| 1:906506      | ["G","A"]  |
| 1:906507      | ["G","A"]  |
| 1:906511      | ["C","T"]  |
| 1:906515      | ["C","T"]  |
| 1:906518      | ["C","T"]  |
+---------------+------------+

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| freq                                                                                                                                                                                                          |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>                                                                                                                                     |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| [(7,3.30e-05,212084,0),(19,7.58e-05,250616,0),(3,3.27e-05,91642,0),(0,0.00e+00,18778,0),(0,0.00e+00,5334,0),(0,0.00e+00,9190,0),(1,8.26e-05,12108,0),(1,6.18e-05,16194,0),(1,3.66e-05,27302,0),(1,3.17e-05... |
| [(3,1.41e-05,213032,0),(14,5.59e-05,250620,0),(2,2.17e-05,92250,0),(0,0.00e+00,18832,0),(0,0.00e+00,5360,0),(0,0.00e+00,9190,0),(0,0.00e+00,12130,0),(0,0.00e+00,16292,0),(1,3.66e-05,27358,0),(0,0.00e+00... |
| [(16,7.46e-05,214396,0),(27,1.08e-04,250632,0),(4,4.30e-05,92998,0),(0,0.00e+00,18892,0),(0,0.00e+00,5374,0),(0,0.00e+00,9212,0),(0,0.00e+00,12246,0),(1,6.10e-05,16394,0),(3,1.09e-04,27492,0),(8,2.52e-0... |
| [(3,1.37e-05,218706,0),(9,3.59e-05,250614,0),(0,0.00e+00,95506,0),(0,0.00e+00,19090,0),(0,0.00e+00,5434,0),(0,0.00e+00,9276,0),(0,0.00e+00,12664,0),(0,0.00e+00,16736,0),(3,1.08e-04,27826,0),(0,0.00e+00,... |
| [(5,2.30e-05,217256,0),(17,6.79e-05,250548,0),(2,2.11e-05,94884,0),(0,0.00e+00,18926,0),(0,0.00e+00,5394,0),(0,0.00e+00,9236,0),(0,0.00e+00,12426,0),(0,0.00e+00,16572,0),(3,1.08e-04,27742,0),(0,0.00e+00... |
| [(13,5.96e-05,218016,0),(24,9.58e-05,250576,0),(12,1.26e-04,95256,0),(0,0.00e+00,18958,0),(0,0.00e+00,5408,0),(0,0.00e+00,9246,0),(0,0.00e+00,12528,0),(0,0.00e+00,16634,0),(1,3.59e-05,27850,0),(0,0.00e+... |
| [(38,1.74e-04,218482,0),(51,2.04e-04,250592,1),(38,3.98e-04,95462,0),(0,0.00e+00,18990,0),(0,0.00e+00,5428,0),(0,0.00e+00,9264,0),(0,0.00e+00,12588,0),(0,0.00e+00,16686,0),(0,0.00e+00,27864,0),(0,0.00e+... |
| [(13,5.94e-05,218732,0),(50,2.00e-04,250624,0),(3,3.14e-05,95664,0),(0,0.00e+00,18960,0),(1,1.84e-04,5430,0),(0,0.00e+00,9278,0),(0,0.00e+00,12628,0),(0,0.00e+00,16672,0),(0,0.00e+00,27866,0),(9,2.79e-0... |
| [(8,3.63e-05,220098,0),(17,6.78e-05,250632,0),(8,8.30e-05,96422,0),(0,0.00e+00,18964,0),(0,0.00e+00,5464,0),(0,0.00e+00,9322,0),(0,0.00e+00,12816,0),(0,0.00e+00,16802,0),(0,0.00e+00,27922,0),(0,0.00e+00... |
| [(1,4.56e-06,219464,0),(12,4.79e-05,250618,0),(0,0.00e+00,96124,0),(0,0.00e+00,18862,0),(0,0.00e+00,5452,0),(0,0.00e+00,9314,0),(0,0.00e+00,12738,0),(1,5.97e-05,16752,0),(0,0.00e+00,27874,0),(0,0.00e+00... |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

+---------------------------------------------------------------------------------------------------------------------------------------------

In [52]:
# test.export('/gstock/EXOTIC/data/VARIATIONS/EXOTIC_variations_raw.hdf5')
# test = test.to_pandas()
# test['id'] = test['id'].str.replace('/', '_')
# test['freq'] = test['freq'].apply(lambda r: r[0])
# test['AC'] = test['freq'].apply(lambda r: r[0])
# test['AF'] = test['freq'].apply(lambda r: r[1])
# test['AN'] = test['freq'].apply(lambda r: r[2])
# test = test.drop(['freq'], axis=1)
# test = test.loc[test['AC'] > 0]

# test_vaex = vaex.from_pandas(test)
# test_vaex
# test

In [29]:
test2 = pd.read_csv('/gstock/EXOTIC/data/VARIATIONS/EXOTIC_variations_raw.hdf5', sep='\t')
test2

,locus,alleles,AF,rsid,variant_type,end,id,most_severe_consequence,MAP
0,1:906494,"[""G"",""A""]",0.000033,rs200703251,snv,906494,1_906494_G/A,missense_variant,PLEKHN1_906493-906588
1,1:906496,"[""C"",""T""]",0.000014,rs547187116,snv,906496,1_906496_C/T,missense_variant,PLEKHN1_906493-906588
2,1:906497,"[""G"",""A""]",0.000075,rs200313046,snv,906497,1_906497_G/A,missense_variant,PLEKHN1_906493-906588
3,1:906503,"[""C"",""A""]",0.000014,rs748042832,snv,906503,1_906503_C/A,missense_variant,PLEKHN1_906493-906588
4,1:906505,"[""C"",""T""]",0.000023,rs368512764,snv,906505,1_906505_C/T,missense_variant,PLEKHN1_906493-906588
...,...,...,...,...,...,...,...,...,...
2414,1:6021972,"[""C"",""A""]",0.000004,rs772023605,snv,6021972,1_6021972_C/A,synonymous_variant,NPHP4_6021854-6022009
2415,1:6021976,"[""C"",""T""]",0.000028,rs371641969,snv,6021976,1_6021976_C/T,missense_variant,NPHP4_6021854-6022009
2416,1:6021982,"[""T"",""A""]",0.000004,rs1206134101,snv,6021982,1_6021982_T/A,missense_variant,NPHP4_6021854-6022009
2417,1:6021988,"[""A"",""G""]",0.000008,rs759671585,snv,6021988,1_6021988_A/G,missense_variant,NPHP4_6021854-6022009


In [ ]:
test2 = pd.read_hdf('/gstock/EXOTIC/data/VARIATIONS/EXOTIC_variations_raw.hdf5')
test2

In [ ]:

import vaex

test2 = vaex.read_csv('/gstock/EXOTIC/data/VARIATIONS/EXOTIC_variations_raw.tsv.bgz')
test2

In [8]:
import vaex
cadd = vaex.open('/gstock/biolo_datasets/variation/CADD/1.6/gnomad.genomes.r2.1.1.snv.parquet')
cadd['id'] = cadd['#Chrom'] + '_' + cadd['Pos'].astype(str) + '_' + cadd['Ref'] + '_' + cadd['Alt']
cadd



#,#Chrom,Pos,Ref,Alt,RawScore,PHRED,variant_id,id
0,'1',10114,'T','C',0.180882,5.5520000000000005,'1_10114_T_C','1_10114_T_C'
1,'1',10120,'T','C',0.125006,4.6739999999999995,'1_10120_T_C','1_10120_T_C'
2,'1',10137,'C','G',-0.014086000000000001,2.416,'1_10137_C_G','1_10137_C_G'
3,'1',10138,'T','C',0.12558,4.683,'1_10138_T_C','1_10138_T_C'
4,'1',10140,'A','G',0.127136,4.709,'1_10140_A_G','1_10140_A_G'
...,...,...,...,...,...,...,...,...
"228,570,207",'X',155260459,'T','G',0.192326,5.726,'X_155260459_T_G','X_155260459_T_G'
"228,570,208",'X',155260460,'T','G',0.192325,5.726,'X_155260460_T_G','X_155260460_T_G'
"228,570,209",'X',155260461,'A','G',0.214802,6.058,'X_155260461_A_G','X_155260461_A_G'
"228,570,210",'X',155260466,'G','T',0.10944000000000001,4.419,'X_155260466_G_T','X_155260466_G_T'


In [54]:
cadd_hail = hl.import_table('/gstock/biolo_datasets/variation/CADD/1.6/gnomad.genomes.r2.1.1.h.snv.tsv.gz', force_bgz=True)
cadd_hail = cadd_hail.annotate(
    id = cadd_hail['#Chrom'] + '_' + cadd_hail['Pos'] + '_' + cadd_hail['Ref'] + '_' + cadd_hail['Alt']
)
cadd_hail = cadd_hail.key_by('id')
cadd_hail.head(10).show()

2021-04-22 19:12:06 Hail: INFO: Reading table without type imputation
  Loading field '#Chrom' as type str (not specified)
  Loading field 'Pos' as type str (not specified)
  Loading field 'Ref' as type str (not specified)
  Loading field 'Alt' as type str (not specified)
  Loading field 'RawScore' as type str (not specified)
  Loading field 'PHRED' as type str (not specified)
2021-04-22 19:14:27 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,,,,
#Chrom,Pos,Ref,Alt,RawScore,PHRED,id
str,str,str,str,str,str,str
"""10""","""10000000""","""C""","""T""","""-0.240651""","""0.547""","""10_10000000_C_T"""
"""10""","""100000012""","""G""","""A""","""-0.263389""","""0.469""","""10_100000012_G_A"""
"""10""","""10000001""","""C""","""T""","""-0.225836""","""0.605""","""10_10000001_C_T"""
"""10""","""10000002""","""G""","""A""","""-0.233573""","""0.574""","""10_10000002_G_A"""
"""10""","""100000031""","""C""","""T""","""-0.006832""","""2.522""","""10_100000031_C_T"""
"""10""","""100000035""","""C""","""T""","""-0.001967""","""2.594""","""10_100000035_C_T"""
"""10""","""100000056""","""C""","""G""","""-0.281881""","""0.414""","""10_100000056_C_G"""
"""10""","""100000101""","""T""","""C""","""0.316442""","""7.423""","""10_100000101_T_C"""


In [57]:
t = test.annotate(
    PHRED = cadd_hail[test.id].PHRED
)
t.head(10).show()

2021-04-22 19:16:35 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-04-22 19:16:36 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-04-22 19:17:08 Hail: INFO: Coerced sorted dataset
2021-04-22 19:17:08 Hail: INFO: Coerced dataset with out-of-order partitions.


KeyboardInterrupt: 

In [ ]:
join = cadd.join(test_vaex, left_on='id', right_on='id', how='inner')
join.export_parquet('/gstock/EXOTIC/data/VARIATIONS/EXOTIC_variations_gnomad.parquet')
# print("--- %s seconds ---" % (time.time() - start_time))


In [101]:
# import pandas as pd
# cadd = pd.read_csv('/gstock/biolo_datasets/variation/CADD/1.6/gnomad.genomes.r2.1.1.snv.tsv.gz', skiprows=1, sep='\t', compression='gzip')
# cadd['variant_id'] = cadd['#Chrom'].astype(str) + '_' + cadd['Pos'].astype(str) + '_' + cadd['Ref'] + '_' + cadd['Alt']
cadd['#Chrom'] = cadd['#Chrom'].astype(str)
# cadd['#Chrom'] = cadd['#Chrom'].astype(str)
cadd.to_parquet('/gstock/biolo_datasets/variation/CADD/1.6/gnomad.genomes.r2.1.1.snv.parquet', index=False)
# cadd.to_hdf('/gstock/biolo_datasets/variation/CADD/1.6/gnomad.genomes.r2.1.1.snv.hdf5', index=False, key='df')

cadd

,#Chrom,Pos,Ref,Alt,RawScore,PHRED,variant_id
0,1,10114,T,C,0.180882,5.552,1_10114_T_C
1,1,10120,T,C,0.125006,4.674,1_10120_T_C
2,1,10137,C,G,-0.014086,2.416,1_10137_C_G
3,1,10138,T,C,0.125580,4.683,1_10138_T_C
4,1,10140,A,G,0.127136,4.709,1_10140_A_G
...,...,...,...,...,...,...,...
228570207,X,155260459,T,G,0.192326,5.726,X_155260459_T_G
228570208,X,155260460,T,G,0.192325,5.726,X_155260460_T_G
228570209,X,155260461,A,G,0.214802,6.058,X_155260461_A_G
228570210,X,155260466,G,T,0.109440,4.419,X_155260466_G_T


In [4]:
test = ['chr1:50799409-chr1:50885666']

intervals = bed_file.annotate(interval_copy = bed_file.interval)
intervals.show()

intervals_raw = [hl.parse_locus_interval('{}'.format(e), reference_genome='GRCh38') for e in test] # or some other way to make a list of
data_lite = hl.filter_intervals(data, intervals_raw)

data_lite = data_lite.filter((data_lite.popmax.AC > 0))
data_lite = data_lite.select(data_lite.popmax.AC, data_lite.popmax.AF, data_lite.vep, data_lite.cadd)

print(data_lite.count())

data_lite = data_lite.annotate(
    interval = intervals[data_lite.locus].interval_copy,
    gene = intervals[data_lite.locus].target,
)
data_lite.tail(1000).show()


2021-04-22 11:56:12 Hail: INFO: Ordering unsorted dataset with network shuffle


,,
interval,target,interval_copy
interval<locus<GRCh38>>,str,interval<locus<GRCh38>>
[chr1:33302016-chr1:33302266),"""Upstream""",[chr1:33302016-chr1:33302266)
[chr1:33302266-chr1:33302516),"""Upstream""",[chr1:33302266-chr1:33302516)
[chr1:33302516-chr1:33302766),"""Upstream""",[chr1:33302516-chr1:33302766)
[chr1:33302766-chr1:33303016),"""Upstream""",[chr1:33302766-chr1:33303016)
[chr1:33303016-chr1:33303266),"""Upstream""",[chr1:33303016-chr1:33303266)
[chr1:33303266-chr1:33303516),"""Upstream""",[chr1:33303266-chr1:33303516)
[chr1:33303516-chr1:33303766),"""Upstream""",[chr1:33303516-chr1:33303766)
[chr1:33303766-chr1:33304016),"""Upstream""",[chr1:33303766-chr1:33304016)


18684


2021-04-22 11:56:19 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-04-22 11:56:20 Hail: INFO: Ordering unsorted dataset with network shuffle


+---------------+-----------------------------------+-------+----------+-------------------+------------------------------+---------------+-------------+----------+--------+
| locus         | alleles                           |    AC |       AF | vep.assembly_name | vep.allele_string            | vep.ancestral | vep.context |  vep.end | vep.id |
+---------------+-----------------------------------+-------+----------+-------------------+------------------------------+---------------+-------------+----------+--------+
| locus<GRCh38> | array<str>                        | int32 |  float64 | str               | str                          | str           | str         |    int32 | str    |
+---------------+-----------------------------------+-------+----------+-------------------+------------------------------+---------------+-------------+----------+--------+
| chr1:50882553 | ["G","T"]                         |    28 | 4.12e-04 | "GRCh38"          | "G/T"                        | NA            | NA          | 50882553 | "."    |
| chr1:50882563 | ["C","T"]                         |     2 | 2.94e-05 | "GRCh38"          | "C/T"                        | NA            | NA          | 50882563 | "."    |
| chr1:50882566 | ["C","T"]                         |     2 | 4.84e-05 | "GRCh38"          | "C/T"                        | NA            | NA          | 50882566 | "."    |
| chr1:50882567 | ["A","C"]                         |     4 | 9.66e-05 | "GRCh38"          | "A/C"                        | NA            | NA          | 50882567 | "."    |
| chr1:50882584 | ["A","T"]                         |     1 | 6.58e-05 | "GRCh38"          | "A/T"                        | NA            | NA          | 50882584 | "."    |
| chr1:50882585 | ["G","C"]                         |     1 | 2.44e-05 | "GRCh38"          | "G/C"                        | NA            | NA          | 50882585 | "."    |
| chr1:50882588 | ["C","A"]                         |     2 | 1.34e-04 | "GRCh38"          | "C/A"                        | NA            | NA          | 50882588 | "."    |
| chr1:50882591 | ["G","C"]                         |     1 | 1.48e-05 | "GRCh38"          | "G/C"                        | NA            | NA          | 50882591 | "."    |
| chr1:50882592 | ["A","T"]                         |     1 | 1.48e-05 | "GRCh38"          | "A/T"                        | NA            | NA          | 50882592 | "."    |
| chr1:50882595 | ["C","G"]                         |     2 | 3.99e-04 | "GRCh38"          | "C/G"                        | NA            | NA          | 50882595 | "."    |
| chr1:50882599 | ["C","A"]                         |     1 | 1.72e-05 | "GRCh38"          | "C/A"                        | NA            | NA          | 50882599 | "."    |
| chr1:50882599 | ["CTTAA","C"]                     |     1 | 2.17e-04 | "GRCh38"          | "TTAA/-"                     | NA            | NA          | 50882603 | "."    |
| chr1:50882600 | ["T","A"]                         |     1 | 2.07e-04 | "GRCh38"          | "T/A"                        | NA            | NA          | 50882600 | "."    |
| chr1:50882600 | ["T","TTAAA"]                     |   446 | 1.05e-01 | "GRCh38"          | "-/TAAA"                     | NA            | NA          | 50882600 | "."    |
| chr1:50882600 | ["T","TTAAATAAA"]                 |   630 | 1.68e-02 | "GRCh38"          | "-/TAAATAAA"                 | NA            | NA          | 50882600 | "."    |
| chr1:50882600 | ["T","TTAAATAAATAAA"]             |     4 | 9.45e-04 | "GRCh38"          | "-/TAAATAAATAAA"             | NA            | NA          | 50882600 | "."    |
| chr1:50882600 | ["TTAAA","T"]                     | 11249 | 3.01e-01 | "GRCh38"          | "TAAA/-"                     | NA            | NA          | 50882604 | "."    |
| chr1:50882600 | ["TTAAATAAA","T"]                 | 10280 | 2.74e-01 | "GRCh38"          | "TAAATAAA/-"                 | NA            | NA          | 50882608 | "."    

In [ ]:
data_lite = data_lite.filter((data_lite > 0))


In [ ]:
import pandas as pd
df = pd.read_parquet('/gstock/EXOTIC/data/VARIATIONS/clinvar_20210123_lite_table.parquet')
df = df.loc[(df['Status'] == 'Benign') & (df['MC'].str.contains('synonymous'))]
df = df.head(100)
df['locus'] = 'chr' + df['CHROM'] + ':' + df.POS.astype(str)
df['alleles'] = df.apply(lambda r: [r['REF'],eval(r['ALT'])[0]], axis=1)
df


In [ ]:
merge_biomart_refseq_38 = hl.import_table("/gstock/EXOTIC/data/GENOMICS/merge_biomart_refseq_38.csv", impute=True, delimiter='\t')
merge_biomart_refseq_38.show()

In [ ]:
data_lite = data_lite.annotate(biomart = merge_biomart_refseq_38[data_lite.].)


In [ ]:
import pandas as pd
# bed = pd.DataFrame([['chr1', 'chr11'], [10031,66510646],[10032,66510647]]).T
bed = pd.DataFrame([['chr1'], [10031],[10031]]).T
# bed = pd.DataFrame([['chr11'], [66510646],[66510647]]).T
bed.to_csv('test2.bed', sep='\t', header=False, index=False)
interval_table = hl.import_bed('test2.bed', reference_genome='GRCh38')
interval_table

In [ ]:
gnomad.utils.filtering.filter_to_autosomes(test)

In [ ]:
test.show()

In [ ]:
data_lite.filter(data_lite.popmax.AF > 1e-2).popmax.show()

In [ ]:

ds_result.show()

In [ ]:
interval = hl.parse_locus_interval('chr11:66510646-66510647', reference_genome='GRCh38')
data_lite = data.filter(interval.contains(data.locus))
data_lite.show()

In [ ]:
filtered_mt = data_lite.filter(hl.is_defined(interval_table[data_lite.locus]))
filtered_mt.show()

In [ ]:
filtered_mt.filter(filtered_mt.popmax.AF > 2.5e-5).to_pandas()